## 1. Create TEASERPLUS project

## 2. Get polygon coordinates for area of interest from QGIS

**Use QGIS field calculator:**
>```python
>geom_to_wkt($geometry)
>```

**Output:**
>MultiPolygon (((1818711.45449641 6145714.52793876, 1818896.19119223 6145696.76479493, 1819169.03308143 6145675.44902233, 1819119.29627871 6145502.79126432, 1819080.21736229 6145357.84401069, 1818892.90839519 6145442.25652424, 1818707.1913419 6145519.8438824, 1818707.1913419 6145519.8438824, 1818711.45449641 6145714.52793876)))

In [1]:
# import modules and functions
from teaser.project import Project
import teaser.data.input.citygml_input as cg
import teaser.data.output.citygml_output as cg_out
import teaser.logic.utilities as utilities
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Create TEASERPLUS project class
prj = Project(load_data=True)
prj.name = 'teaser'

## 2. Load CityGML files

- Calculate building geometries
- Calculate building physics properties

In [3]:
current_dir = os.getcwd()
path_to_gml = os.path.join(current_dir, 'data', 'citygml_files', 'citydb_export.gml')

gml = cg.load_gml_lxml(path_to_gml, prj, method='iwu', calc_sep=False)

B1180Sternwartestrae52 bldg.set_gml_attributes() did work
B1180Sternwartestrae52 bldg.set_gml_attributes() did work
B1180AnastasiusGrnGasse52_2 bldg.set_gml_attributes() did work
B1180AnastasiusGrnGasse52_2 bldg.set_gml_attributes() did work
B1180GustavTschermakGasse6 bldg.set_gml_attributes() did work
B1180GustavTschermakGasse6 bldg.set_gml_attributes() did work
B1180GregorMendelStrae28_1 bldg.set_gml_attributes() did work
B1180GregorMendelStrae28_1 bldg.generate_from_gml() did work
B1180Cottagegasse29 bldg.set_gml_attributes() did work
B1180Cottagegasse29 bldg.set_gml_attributes() did work
B1180GustavTschermakGasse4 bldg.set_gml_attributes() did work
B1180GustavTschermakGasse4 bldg.set_gml_attributes() did work
B1180AnastasiusGrnGasse58 bldg.set_gml_attributes() did work
B1180AnastasiusGrnGasse58 bldg.set_gml_attributes() did work
B1180Colloredogasse33A_1 bldg.set_gml_attributes() did work
B1180Colloredogasse33A_1 bldg.set_gml_attributes() did work
B1180Colloredogasse29 bldg.set_gml_

## 3. Calculate teaser thermal zones and buildings and export

In [4]:
# path to weather file (converted from epw to mos)
path_to_weather = os.path.join(current_dir, 'data', 'weather_files', 'AUT_WI_Wien_110800_TMYx_2021.mos')

# path for modelica output
path_to_output = os.path.join(current_dir, 'output', 'modelica')

# number of building elements (4 = roof, floor, wall, windows)
prj.number_of_elements_calc = 4

# choose modelica library (IBPSA, AixLib) (IBPSA -> faster, AixLib -> slow, alot of building systems)
prj.used_library_calc ='IBPSA'

# specify weatherfile
prj.weather_file_path = path_to_weather

# calculate all building properties
prj.calc_all_buildings(raise_errors=True)

# export modelica models
prj.export_ibpsa(library='AixLib', internal_id=None, path=path_to_output)

Exports can be found here:
c:\Users\dawo\e7dw\MAA\simulations\06_demo_new\demo\output\modelica\teaser


'c:\\Users\\dawo\\e7dw\\MAA\\simulations\\06_demo_new\\demo\\output\\modelica\\teaser'

## 4. Edit modelica files

**Building models do not have heating system**

- Add simple heating system
    - Add ideal heater
    - Add PI controler (Setpoint indoor air temperature = 22°C)
    - Add output connector QFlowHeat from PI controller

In [6]:
# import modules
from dhc_sim.utils import find_files_by_end_of_filename
from dhc_sim.processing.modelica import add_pid_heater_to_IBPSA_model
from modelica_builder.model import Model

# path to modelica folder
modelica_path = os.path.join(current_dir, 'output', 'modelica')

# find all modelica building models
files = find_files_by_end_of_filename(modelica_path, 'SingleDwelling.mo')

# edit each building model
for i, file in enumerate(files):
    file_name = file['file_name']
    file_path = file['path']
    
    # create model class (modelica builder)
    model = Model(file_path)
    
    # add pid controlled heater
    model = add_pid_heater_to_IBPSA_model(model)

    # save edited model
    model.save_as(file_path)
    
    print(f"[{i+1}/{len(files)}] {file_name} successfully modified")

ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtime and generated code versions disagree: 4.13.1!=4.11.2-SNAPSHOT
ANTLR runtim

# 5. Export TEASERPLUS project as pickle

In [7]:
# Export prj to export TEASERPLUS Project class
import pickle

pickle_file = os.path.join(current_dir, "output", "demo_teaser_pickle.p")

pickle.dump(prj, open(pickle_file, "wb"))